## *C elegans* basic info

Verify the number of neurons and connections.

Neuron connection data is from [here](http://wormweb.org/data/NeuronConnect.txt).

Neuron name/group/type data is from [here](http://wormweb.org/data/name_neurons.txt).

"Old" data is from [here](https://github.com/openworm/CElegansNeuroML/blob/master/CElegansNeuronTables.xls).

Other links:

* [Visualization page](http://wormweb.org/neuralnet)
* [Details page](http://wormweb.org/details.html)
* [Basic information](http://nikhil.superfacts.org/archives/2009/04/the_neural_netw.html)
* [More details on neurons, including explanation of CANL, CANR and VC6](https://wormatlas.org/neuronalwiring.html#NeuronalconnectivityII)

In [1]:
import pandas as pd

### Read `name_neurons.txt` with neuron names, groups and types

In [2]:
with open("name_neurons.txt", encoding="utf-8") as f:
    nn_text = [s.strip() for s in f.readlines()]

In [3]:
nn_recs = [s.split() for s in nn_text if not(s.startswith("#"))]
df_nn = pd.DataFrame({
    "name": [nr[0] for nr in nn_recs], 
    "group": [nr[1] for nr in nn_recs], 
    "type": [nr[2] for nr in nn_recs]
})
df_nn.head()

,name,group,type
0,ADAL,ADA,in
1,ADAR,ADA,in
2,ADEL,ADE,se
3,ADER,ADE,se
4,ADFL,ADF,se


In [4]:
df_nn.to_csv("name_neurons.csv", encoding="utf-8", index=False)

### Read `NeuronConnect.txt` with connections

In [5]:
with open("NeuronConnect.txt", encoding="utf-8") as f:
    nc_text = [s.strip() for s in f.readlines()]

In [6]:
nc_recs = [s.split() for s in nc_text if not(s.startswith("#"))]
df_nc = pd.DataFrame({
    "neuron_1": [nr[0] for nr in nc_recs],
    "neuron_2": [nr[1] for nr in nc_recs],
    "type": [nr[2] for nr in nc_recs],
    "nbr": [int(nr[3]) for nr in nc_recs]
})
df_nc.head()

,neuron_1,neuron_2,type,nbr
0,ADAR,ADAL,EJ,1
1,ADFL,ADAL,EJ,1
2,ASHL,ADAL,EJ,1
3,AVDR,ADAL,EJ,2
4,PVQL,ADAL,EJ,1


In [7]:
df_nc.to_csv("NeuronConnect.csv", encoding="utf-8", index=False)

### Read `synapse_AF.txt` with pharnyx network connections

In [8]:
with open("synapse_AF.txt", encoding="utf-8") as f:
    pn_text = [s.strip() for s in f.readlines()]

In [9]:
pn_recs = [s.split() for s in pn_text if not(s.startswith("#"))]
df_pn = pd.DataFrame({
    "neuron_1": [pn[0] for pn in pn_recs],
    "type_1": [pn[1] for pn in pn_recs],
    "nbr_1": [int(pn[2]) for pn in pn_recs],
    "nbr_2": [int(pn[3]) for pn in pn_recs],
    "type_2": [pn[4] for pn in pn_recs],
    "neuron_2": [pn[5] for pn in pn_recs],
    "v": [pn[6] for pn in pn_recs],
    "src": [pn[7] for pn in pn_recs]
})
df_pn.head()

,neuron_1,type_1,nbr_1,nbr_2,type_2,neuron_2,v,src
0,I1L,A,1,1,G,RIP,-,-
1,I1L,B,2,2,S,M2,-,-
2,I1L,B,2,2,S,M3,-,-
3,I1L,B,3,3,S,MC,-,-
4,I1L,B,4,4,S,I5,v,-


### Which name_neurons values are actually neurons?

```
se = sensory neuron
in = interneuron
mo = motorneuron
mu = muscle
bm = basement membrane
gln = gland cell
mc = marginal cell
```

In [10]:
df_nn["type"].value_counts()

mo       92
in       90
se       71
mo/in    25
mu        8
in/mo     7
se/mo     6
mo/se     4
in/se     4
se/in     3
gln       2
mc        2
bm        1
Name: type, dtype: int64

In [11]:
df_nn_neurons = df_nn[df_nn["type"].isin(["mo", "in", "se", "mo/in", "in/mo", "se/mo", "mo/se", "in/se", "se/in"])]
neuron_names = set(df_nn_neurons["name"])
df_nn_neurons.shape

(302, 3)

**Conclusion:** There are 302 values in the neuron name list that correspond to actual neurons, just as expected.

### Which name_neurons values do not show up in any connections?

In [12]:
nc_names = set(df_nc["neuron_1"]) | set(df_nc["neuron_2"])
print("len(nc_names) =", len(nc_names))

len(nc_names) = 283


In [13]:
df_nn_neurons[(~df_nn_neurons["name"].isin(nc_names)) & (~df_nn_neurons["group"].isin(nc_names))].sort_values(by=["name"])

,name,group,type
82,CANL,CAN,in
83,CANR,CAN,in
119,I2L,I2,in
120,I2R,I2,in
121,I3,I3,in
122,I4,I4,in
123,I5,I5,in
124,I6,I6,in
139,M1,M1,mo
140,M2L,M2,mo


In [14]:
nc_names - set(df_nn_neurons["name"])

{'NMJ'}

There are 20 neurons that do not have any connections listed. There is one name that occurs in the connection data but not the neuron list: "NMJ". According to [this source](https://wormatlas.org/neuronalwiring.html#NeuronalconnectivityII) these are "neuromuscular junctions", so should be discounted. All of the connections listing "NMJ" as `neuron_1` or `neuron_2` are of type "NMJ". 

To be safe, let's verify that the only connections with `neuron_1` or `neuron_2` values of "NMJ" are also of type "NMJ":

In [15]:
df_nc[~(df_nc["type"] == "NMJ") & ((df_nc["neuron_1"] == "NMJ") | (df_nc["neuron_2"] == "NMJ"))].shape

(0, 4)

### Do all pharnyx neurons occur in the neuron name list?

In [16]:
nn_names = set(df_nn_neurons["name"])
nn_groups = set(df_nn_neurons["group"])
nn_all = nn_names | nn_groups
df_pn[~df_pn["neuron_2"].isin(nn_all)]

,neuron_1,type_1,nbr_1,nbr_2,type_2,neuron_2,v,src
70,I5,C,9,11,S,G1,-,-
71,I5,C,10,12,M,PM5,-,-
81,I5,H,18,21,M,PM5,-,-
82,I5,H,18,21,M,G1,-,-
92,M1,A,1,1,M,PM3,-,-
...,...,...,...,...,...,...,...,...
509,NSMR,B,32,32,M,PM5,-,-
510,NSMR,B,33,33,M,PM5,-,-
511,NSMR,B,34,34,M,PM5,-,-
513,NSMR,B,36,36,M,PM5,-,af8


**Answer:** All `neuron_1` names appear in the list, but 301 records have `neuron_2` names which do not. What are the distinct values with counts? What are the distinct values that *do* occur, with counts?

In [17]:
df_pn[~df_pn["neuron_2"].isin(nn_all)]["neuron_2"].value_counts().sort_index()

BM       33
G         2
G1       22
PM1       8
PM2       4
PM3       4
PM4      21
PM5     155
PM6      10
PM7      32
PMC       6
PMC2      4
Name: neuron_2, dtype: int64

In [18]:
df_pn[df_pn["neuron_2"].isin(nn_all)]["neuron_2"].value_counts().sort_index()

I1     17
I1L     1
I1R     1
I2     18
I3     16
I4     15
I5     20
I6     11
M1     19
M2     16
M2L     1
M2R     1
M3     16
M3L     1
M3R     1
M4     18
M5     13
MC      9
MI      4
NSM    20
RIP     2
Name: neuron_2, dtype: int64

What are the distinct values in `neuron_1`?

In [19]:
df_pn["neuron_1"].value_counts().sort_index()

I1L      12
I1R      11
I2L       8
I2R       8
I3        7
I4       12
I5       26
I6        8
M1       32
M2L      37
M2R      40
M3L      18
M3R      17
M4      105
M5       63
MCL       7
MCR       6
MI       20
NSML     38
NSMR     46
Name: neuron_1, dtype: int64

Compare this to the list of neurons names that do not show up in the connections.

In [20]:
df_nn_neurons[(~df_nn_neurons["name"].isin(nc_names)) & (~df_nn_neurons["group"].isin(nc_names))].sort_values(by=["name"])

,name,group,type
82,CANL,CAN,in
83,CANR,CAN,in
119,I2L,I2,in
120,I2R,I2,in
121,I3,I3,in
122,I4,I4,in
123,I5,I5,in
124,I6,I6,in
139,M1,M1,mo
140,M2L,M2,mo


In [21]:
ncon_names = set(df_nn_neurons[(~df_nn_neurons["name"].isin(nc_names)) & (~df_nn_neurons["group"].isin(nc_names))]["name"])
pn_names = set(df_pn["neuron_1"])

ncon_names - pn_names, pn_names - ncon_names

({'CANL', 'CANR'}, {'I1L', 'I1R'})

`CANL` and `CANR` are the names of two neurons which are known not to have any connections to other neurons. `I1L` and `I1R` correspond to `I1` which is the only neuron connecting the pharangeal system to the remainder of the nervous system.

### Compare to data from GitHub repository

Check this list of connections (including both sets) against data obtained from [this GitHub repository](https://github.com/openworm/CElegansNeuroML).

In [22]:
df_old = pd.read_csv("CElegansNeuronTables-connectome.csv")
df_old.head()

,Origin,Target,Type,Number of Connections,Neurotransmitter
0,ADAL,ADAR,GapJunction,1,Generic_GJ
1,ADAL,ADFL,GapJunction,1,Generic_GJ
2,ADAL,AIBL,Send,1,Glutamate
3,ADAL,AIBR,Send,2,Glutamate
4,ADAL,ASHL,GapJunction,1,Generic_GJ


In [23]:
def n_tuple(n1, n2):
    if n1 < n2:
        return (n1, n2)
    return (n2, n1)

def nc_key(row):
    return n_tuple(row["neuron_1"], row["neuron_2"])

def old_key(row):
    return n_tuple(row["Origin"], row["Target"])

new_keys_nn = set(df_nc[(df_nc["neuron_1"].isin(neuron_names)) & (df_nc["neuron_2"].isin(neuron_names))].apply(nc_key, axis=1))
new_keys_pn = set(df_pn[(df_pn["neuron_1"].isin(neuron_names)) & (df_pn["neuron_2"].isin(neuron_names))].apply(nc_key, axis=1))
new_keys = new_keys_nn | new_keys_pn

old_keys = set(df_old.apply(old_key, axis=1))

new_keys - old_keys, old_keys - new_keys

({('I3', 'M3L'), ('I3', 'MI'), ('I4', 'MI'), ('M1', 'M4'), ('M1', 'MI')},
 {('I1L', 'I2L'),
  ('I1L', 'I2R'),
  ('I1L', 'I3'),
  ('I1L', 'M2L'),
  ('I1L', 'M2R'),
  ('I1L', 'M3L'),
  ('I1L', 'M3R'),
  ('I1L', 'MCL'),
  ('I1L', 'MCR'),
  ('I1L', 'NSML'),
  ('I1L', 'NSMR'),
  ('I1R', 'I2L'),
  ('I1R', 'I2R'),
  ('I1R', 'I3'),
  ('I1R', 'M2L'),
  ('I1R', 'M2R'),
  ('I1R', 'M3L'),
  ('I1R', 'M3R'),
  ('I1R', 'MCL'),
  ('I1R', 'MCR'),
  ('I1R', 'NSML'),
  ('I1R', 'NSMR'),
  ('I2L', 'MCL'),
  ('I2L', 'NSML'),
  ('I2L', 'NSMR'),
  ('I2R', 'MCL'),
  ('I2R', 'NSML'),
  ('I2R', 'NSMR'),
  ('I3', 'MCL'),
  ('I3', 'MCR'),
  ('I5', 'NSML'),
  ('I5', 'NSMR'),
  ('M2L', 'M4'),
  ('M2L', 'MCL'),
  ('M2L', 'MCR'),
  ('M2R', 'M3L'),
  ('M2R', 'M3R'),
  ('M2R', 'M4'),
  ('M2R', 'MCL'),
  ('M2R', 'MCR'),
  ('M3L', 'MCL'),
  ('M3L', 'MCR'),
  ('M3L', 'MI'),
  ('M3L', 'NSML'),
  ('M3L', 'NSMR'),
  ('M3R', 'MCL'),
  ('M3R', 'MCR'),
  ('M3R', 'MI'),
  ('M3R', 'NSML'),
  ('M3R', 'NSMR'),
  ('M4', 'NSML'),
  ('

When both sets of connections are reduced by eliminating duplicates and discounting the order of the neurons mentioned, and non-neuron names are left out, the only differences occur due to some connections being listed as incident to a group in one set and an individual neuron in the second.

In [24]:
len(old_keys)

2395

In [25]:
pn_names = (set(df_pn["neuron_1"]) | set(df_pn["neuron_2"]))
print(len([x for x in old_keys if x[0] not in pn_names and x[1] not in pn_names]))
print(len([x for x in new_keys if x[0] not in pn_names and x[1] not in pn_names]))

2290
2290


All of these differences occur within the pharyngeal connections list.

In [26]:
final_edges = [x for x in new_keys if x[0] not in pn_names and x[1] not in pn_names]
with open("non-pharyngeal-edges.csv", "w", encoding="utf-8") as f:
    for n1, n2 in final_edges:
        f.write("{},{}\n".format(n1, n2))

### Degree distribution

In [27]:
from collections import Counter
ctr = Counter([e[0] for e in final_edges] + [e[1] for e in final_edges])
deg = Counter(ctr.values())
deg_list = sorted(deg.keys())
degree_dist = [(d, deg[d]) for d in deg_list]

In [28]:
degree_dist[:10]

[(2, 5),
 (3, 3),
 (4, 4),
 (5, 4),
 (6, 13),
 (7, 15),
 (8, 14),
 (9, 17),
 (10, 21),
 (11, 12)]

In [29]:
with open("degree-distribution.csv", "w", encoding="utf-8") as f:
    f.write("Degree,Frequency\n")
    for d, v in degree_dist:
        f.write("{},{}\n".format(d, v))

### Mean distance

In [30]:
import igraph

In [31]:
all_vertices = sorted(set([x[0] for x in final_edges] + [x[1] for x in final_edges]))
g = igraph.Graph()
g.add_vertices(all_vertices)
g.add_edges(final_edges)

In [32]:
print("Mean distance = ", g.average_path_length(directed=False))
print("Total distance = ", g.average_path_length(directed=False) * (len(all_vertices) * (len(all_vertices) - 1) / 2))

Mean distance =  2.435625692993992
Total distance =  94456.0


### Verify number of vertices

In [33]:
len(all_vertices)

279

In [34]:
set(df_nn_neurons["name"]) - set(all_vertices) - pn_names

{'CANL', 'CANR', 'VC6'}

In [35]:
df_nc[(df_nc["neuron_1"] == "VC6") | (df_nc["neuron_2"] == "VC6")]

,neuron_1,neuron_2,type,nbr
6403,VC6,NMJ,NMJ,1


According to [WormAtlas](https://wormatlas.org/neuronalwiring.html#NeuronalconnectivityII), "Along with CANL/R, VC6 was omitted since it only makes one NMJ".

### Mean degree and density

In [36]:
print("Mean degree = ", len(final_edges), " / ", len(all_vertices), " = ", len(final_edges) / len(all_vertices))

Mean degree =  2290  /  279  =  8.207885304659499


In [37]:
print("Density = ", len(final_edges), " / (", len(all_vertices), " * ", len(all_vertices) - 1, ") = ", len(final_edges) / (len(all_vertices) * (len(all_vertices) - 1)))

Density =  2290  / ( 279  *  278 ) =  0.029524767282947836


### Treat the graph as directed and redo the statistics

In [38]:
old_directed_edges = list(set(df_old.apply(lambda row: (row["Origin"], row["Target"]), axis=1)))
old_directed_edges = [x for x in old_directed_edges if x[0] not in pn_names and x[1] not in pn_names]
print("Number of edges (directed): ", len(old_directed_edges))

Number of edges (directed):  2993


In [39]:
with open("non-pharyngeal-directed-edges.csv", "w", encoding="utf-8") as f:
    for e in old_directed_edges:
        f.write("{},{}\n".format(e[0], e[1]))

In [40]:
old_directed_vertices = sorted(set([x[0] for x in old_directed_edges] + [x[1] for x in old_directed_edges]))
gd = igraph.Graph(directed=True)
gd.add_vertices(old_directed_vertices)
gd.add_edges(old_directed_edges)

In [41]:
md = gd.average_path_length(directed=True)
vc = len(old_directed_vertices)
ec = len(old_directed_edges)
v2 = len(old_directed_vertices) * (len(old_directed_vertices) - 1) / 2
print("Number of vertices = ", vc)
print("Number of edges = ", ec)
print("Mean distance = ", md)
print("Total distance = ", md * v2)
print("Mean degree = ", ec, " / ", vc, " = ", float(ec) / vc)
print("Density = ", ec, " / (", vc, " * ", vc - 1, ") = ", ec / (vc * (vc - 1)))

Number of vertices =  279
Number of edges =  2993
Mean distance =  2.876220856962823
Total distance =  111542.72105387524
Mean degree =  2993  /  279  =  10.727598566308243
Density =  2993  / ( 279  *  278 ) =  0.0385884840514685


In [42]:
out_degrees = Counter((x[0] for x in old_directed_edges))
out_deg_dist = sorted(Counter(out_degrees.values()).items(), key=lambda t: t[0])

in_degrees = Counter((x[1] for x in old_directed_edges))
in_deg_dist = sorted(Counter(in_degrees.values()).items(), key=lambda t: t[0])

out_deg_dist[:10], in_deg_dist[:10]

([(1, 6),
  (2, 12),
  (3, 20),
  (4, 13),
  (5, 20),
  (6, 23),
  (7, 15),
  (8, 25),
  (9, 22),
  (10, 14)],
 [(1, 2),
  (2, 16),
  (3, 18),
  (4, 16),
  (5, 19),
  (6, 18),
  (7, 25),
  (8, 26),
  (9, 18),
  (10, 23)])

In [43]:
with open("out-degree-dist.csv", "w", encoding="utf-8") as f:
    f.write("Degree,Frequency\n")
    for t in out_deg_dist:
        f.write("{},{}\n".format(t[0], t[1]))
        
with open("in-degree-dist.csv", "w", encoding="utf-8") as f:
    f.write("Degree,Frequency\n")
    for t in in_deg_dist:
        f.write("{},{}\n".format(t[0], t[1]))


### Treat new edges as directed and compute statistics

In [44]:
new_directed_edges = list(set(df_nc.apply(lambda row: (row["neuron_1"], row["neuron_2"]), axis=1)))
new_directed_edges = [x for x in new_directed_edges if x[0] not in pn_names and x[1] not in pn_names and x[0] in neuron_names and x[1] in neuron_names]
print("Number of edges (new data, directed): ", len(new_directed_edges))

Number of edges (new data, directed):  4577


In [45]:
print("Number of directed edges in new set but not old: ", len(set(new_directed_edges) - set(old_directed_edges)))

Number of directed edges in new set but not old:  1584


There are a lot more edges in the new set, treated as directed, than in the old one. Before proceeding with statistics it is a good idea to find out why.

### How many directed edges in the old and new sets have the opposite edge also occurring?

In [46]:
len(old_directed_edges), sum([1 if (y, x) in old_directed_edges else 0 for x, y in old_directed_edges])

(2993, 1409)

In [47]:
len(new_directed_edges), sum([1 if (y, x) in new_directed_edges else 0 for x, y in new_directed_edges])

(4577, 4577)

In [48]:
len(new_directed_edges)

4577

Since every edge in the new set also has the same edge in the opposite direction, and the total number of edges is odd, there must be one or more loops. Let's find them.

In [49]:
s = set([(x, y) for x, y in new_directed_edges if x == y])
df_nc[df_nc.apply(lambda row: (row["neuron_1"], row["neuron_2"]) in s, axis=1)]

,neuron_1,neuron_2,type,nbr
4234,RIBL,RIBL,EJ,1
4282,RIBR,RIBR,EJ,1
5746,VA8,VA8,EJ,1


According to [Worm Atlas](https://wormatlas.org/neuronalwiring.html#NeuronalconnectivityII) the types are as follows:

> **Type:** Type of synapse: **S:** Send or output (Neuron 1 pre-synaptic to Neuron 2); **Sp:** Send-poly (Neuron 1 is pre-synaptic to more than one postsynaptic partner.  Neuron 2 is just one of these post-synaptic neurons, see Figure 1 below.  In White et al, 1986, these polyadic synaptic connections were denoted by “m” in the tables of Appendix 1); **R:** Receive or input (Neuron 1 is post-synaptic to Neuron 2); **Rp:** Receive-poly (Neuron 1 is one of several post-synaptic partners of Neuron 2.  See Figure 1 and above); **EJ:** Electric junction; **NMJ:** Neuromuscular junction (only reconstructed NMJ's are represented).

This indicates that edges of type "R" and "Rp" in the new set might be reversals of "S" and "Sp" edges. Let's see what happens if we restrict the new directed edges to types "S", "Sp" and "EJ".

In [50]:
new_directed_edges_2 = list(set(df_nc[df_nc["type"].isin(["S", "Sp", "EJ"])].apply(lambda row: (row["neuron_1"], row["neuron_2"]), axis=1)))
new_directed_edges = [x for x in new_directed_edges_2 if x[0] not in pn_names and x[1] not in pn_names and x[0] in neuron_names and x[1] in neuron_names]
print("Number of edges (new data, directed): ", len(new_directed_edges_2))

Number of edges (new data, directed):  2997


Now the old and new edge counts nearly match - difference is only four edges. What are they?

In [51]:
set(new_directed_edges_2) - set(old_directed_edges)

{('RIPL', 'I1L'), ('RIPL', 'I1R'), ('RIPR', 'I1L'), ('RIPR', 'I1R')}

In [52]:
s = set(new_directed_edges_2) - set(old_directed_edges)
df_nc[df_nc.apply(lambda row: (row["neuron_1"], row["neuron_2"]) in s, axis=1)]

,neuron_1,neuron_2,type,nbr
6417,RIPL,I1L,EJ,1
6418,RIPR,I1L,EJ,1
6419,RIPL,I1R,EJ,1
6420,RIPR,I1R,EJ,1


The old data set treated "I1L" and "I2L" as part of the pharyngeal system and therefore excluded them, while the new data set includes them so that the pharyngeal system can be visualized along with the rest. This comment occurs at the bottom of the new data set:

```
# Added to connect to Albertson pharyngeal data
RIPL 	I1L	EJ	1
RIPR	I1L	EJ	1
RIPL	I1R	EJ	1
RIPR	I1R	EJ	1
```

These four connections are also the only ones not listed in the data [connected to this site](https://wormatlas.org/neuronalwiring.html#NeuronalconnectivityII). Note that the numbering in that spreadsheet used "01", "02", etc.

**Conclusion:** The two data sets are consistent, so the statistics based on the older data set are valid.

## Save the directed graph as a named edge list, and also with parallel edges and loops removed

In [54]:
gd.write_ncol("c-elegans.ncol")
gd.simplify()
gd.write_ncol("c-elegans-simple.ncol")

/home/rkingan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: weights attribute does not exists at src/io/ncol.c:265
  """Entry point for launching an IPython kernel.
/home/rkingan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: weights attribute does not exists at src/io/ncol.c:265
  This is separate from the ipykernel package so we can avoid doing imports until
